# !!!! WILL BE UPDATED FOR 0.1.0 !!!!

In [1]:
import pandas as pd
import numpy as np
import os
import sys

# `pip install sysyphus` to use this package
import sysyphus

from dotenv import load_dotenv

sys.path.append("../")

load_dotenv()

# Setting default DPI, pulling it from dotenv if it exists, setting it on 100 if not

try:
    pc_dpi = int(os.getenv('DPI'))
except TypeError:
    pc_dpi = 100
if pc_dpi is None:
    pc_dpi = 100


# 1 - Initializing the boulder object :
- create an object of the Boulder class, this object is the main element we will use to gather and manipulate data
- on initialisation, sysyphus downloads a dataset which contains all of the meteorites of the MetBull database, it is updated monthly via a `cron job` on GitHub - Actions
- the default option of the loaded data is json (for inter language compatibility), but should the json be compromised or incomplete, the pickle file (which is a python specific format) is also available, the parameter should, in this case, be set to `use_json=False`


In [2]:
boulder = sysyphus.Boulder(use_json=True)


cnx: OK
remote content: Loaded


The init method of the Boulder class checks for internet connection and tries to load the remote content.
`cnx: OK` and `remote content: Loaded` are here to confirm that these steps were executed without any issues.
<br><br>
Should there be an issue, it would be displayed like so :
`ConnectionError: The application has no access to the internet`


In [ ]:
# Expected output when no internet cnx (_ = throwaway variable)
_ = sysyphus.Boulder(use_json=True)


<hr>

We can already see the dataset which represent what would be like an address book : it holds some informations but cannot go into details without further requests :

In [3]:
display("HEAD:", boulder.sy_df.head(n=5))
display("TAIL:", boulder.sy_df.tail(n=5))


'HEAD:'

,name,year,country,type,mass,URL,numeric_id
0,Denader 001,2022,Mali,H4-melt breccia,5330.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,1
1,Hassi Khebi 001,2022,Algeria,C3-ung,500.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,1
2,Qaen 001,2016,Iran,L6,21000.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,1
3,Aachen,1880,Germany,L5,21.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,<NA>
4,Aammiq,2000,Lebanon,H6,596.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,<NA>


'TAIL:'

,name,year,country,type,mass,URL,numeric_id
81232,Zsadany,1875,Romania,H5,552.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,<NA>
81233,Zubkovsky,2003,Russia,L6,2170.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,<NA>
81234,Zulu Queen,1976,USA,L3.7,200.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,<NA>
81235,Zvonkov,1955,Ukraine,H6,2570.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,<NA>
81236,Erg Tellis 001,2021,Algeria,H4,6790.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,1


## 1.1 - About these informations :
We can have :
- the name of the meteorite (it's full name, with the numeric identifier)
- the year of the fall or the find
- the country it fell in
- its type (I tried to harmonize it a bit, but it's limited)
- its mass (g)
- its url, which will be used to perform the requests
- its numeric identifier (ex: for `Erg Tellis 001` its `1`), this simplifies filtering later

# 2 - Making a search:
- The prompt will ask for the following parameters :
    - namespace: the name of the meteorite or meteorite group (Yamato, Catalina), no need for digits yet
    - numeric_range: either an exact match, or a range (inclusive) of integers like `123`, if range, tuple of integers like `40,45` - the leading digits don't matter (so Yamato 0000[...]1 == Yamato 1)
    - the country where the meteorite(s) fell in: a string # VALIDATION FN MISSING
    - the type of the meteorite(s) # VALIDATION FN MISSING
- The parameters can be left blank
- The search is error friendly, an invalid prompt will trigger an explicit error message and invite the user to try again
- The text prompts are case insensitive
- the Boulder.make_search() method has a parameter that allows to preview the results, set to True by default : `verbose_results` - set this to False to disable that feature

In [4]:
# I am going to type : `Catalina` , `500,550` and chile :
boulder.make_search()


Refine your search. Press Enter to skip any criterion.


,name,year,country,type,mass,URL,numeric_id
8297,Catalina 500,2022,Chile,Mesosiderite,999.00,https://www.lpi.usra.edu/meteor/metbull.php?co...,500
8298,Catalina 501,2022,Chile,H5,18.80,https://www.lpi.usra.edu/meteor/metbull.php?co...,501
8299,Catalina 502,2022,Chile,LL5,24.30,https://www.lpi.usra.edu/meteor/metbull.php?co...,502
8300,Catalina 503,2017,Chile,H6,29.30,https://www.lpi.usra.edu/meteor/metbull.php?co...,503
8301,Catalina 504,2019,Chile,H5,24.50,https://www.lpi.usra.edu/meteor/metbull.php?co...,504
8302,Catalina 505,2019,Chile,H5,97.80,https://www.lpi.usra.edu/meteor/metbull.php?co...,505
8303,Catalina 506,2019,Chile,H6,20.70,https://www.lpi.usra.edu/meteor/metbull.php?co...,506
8304,Catalina 507,2017,Chile,H5,20.50,https://www.lpi.usra.edu/meteor/metbull.php?co...,507
8305,Catalina 508,2019,Chile,L6,327.00,https://www.lpi.usra.edu/meteor/metbull.php?co...,508
8306,Catalina 509,2019,Chile,L6,37.20,https://www.lpi.usra.edu/meteor/metbull.php?co...,509


## 2.1 - Search results :
- As expected, we have a subset containing 50 meteorites from catalina 500 to 550 inclusive
- The verbosity was left True by default
- We can actually perform further selection on this dataset by accessing its pandas.Dataframe and performing modifications directly on it beofre doing any requests

In [1]:
ids_to_rm = [22, 30, 42]
